In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta


class DataGenerator:
    def __init__(self, tickers, start_date, end_date, seed=42):
        self.tickers = tickers
        self.dates = pd.date_range(start_date, end_date, freq='B')
        np.random.seed(seed)
    
    def generate_price_data(self):
        return pd.DataFrame(
            {ticker: 100 * np.exp(np.cumsum(np.random.normal(0.0002, 0.01, len(self.dates))))
             for ticker in self.tickers},
            index=self.dates
        )
    
    def generate_events(self):
        event_dates = pd.date_range(self.dates[0], self.dates[-1], freq='3M')
        events = []
        for ticker in self.tickers:
            for ed in event_dates:
                events.append({
                    'event_date': ed,
                    'ticker': ticker,
                    'event_type': np.random.choice(['Earnings', 'Filing']),
                    'action': np.random.choice([-1, 0, 1], p=[0.3, 0.4, 0.3])
                })
        
        events_df = pd.DataFrame(events)
        events_df.sort_values(by=['event_date', 'ticker'], inplace=True)
        return events_df.reset_index(drop=True)

class EventDrivenBacktest:
    def __init__(self, price_data, events_df, initial_weights=None, holding_period=5):
        """
        price_data: DataFrame indexed by date with columns for each ticker.
        events_df: DataFrame with columns [event_date, ticker, event_type, action].
        initial_weights: Dict {ticker: weight}. If None, use equal weight.
        holding_period: Number of business days to hold after an event.
        """
        self.price_data = price_data
        self.events_df = events_df.copy()
        self.holding_period = holding_period
        
        # Set initial portfolio weights: if not provided, use equal weight.
        self.weights = self._initialize_weights(initial_weights)

        # Initialize a DataFrame to hold portfolio performance
        self.portfolio_history = self._initialize_portfolio_history()

    def _initialize_weights(self, initial_weights):
        if initial_weights is None:
            return {ticker: 1.0/len(self.price_data.columns) for ticker in self.price_data.columns}
        return initial_weights
    
    def _initialize_portfolio_history(self):
        history = pd.DataFrame(index=self.price_data.index)
        history['portfolio_value'] = np.nan
        history.iloc[0, history.columns.get_loc('portfolio_value')] = 1.0
        return history

    def run(self):
        # Start backtest simulation from the first event date
        # We assume the portfolio is rebalanced at each event (only for the stock with an event)
        current_value = 1.0
        last_date = self.price_data.index[0]
        portfolio_weights = self.weights.copy()
        
        # For each event in chronological order
        for idx, event in self.events_df.iterrows():
            event_date = event['event_date']
            ticker = event['ticker']
            action = event['action']
            
            # Skip events if event_date is not in our price_data index
            if event_date not in self.price_data.index:
                continue
            
            # Simulate performance from last_date up to event_date (if there is a gap)
            # We assume buy-and-hold between events
            period = self.price_data.loc[last_date:event_date]
            if not period.empty:
                # Calculate portfolio return as weighted sum of individual returns
                returns = period.pct_change().fillna(0)
                daily_portfolio_return = returns.dot(pd.Series(portfolio_weights))
                cum_return = np.prod(1 + daily_portfolio_return)  # cumulative return over period
                current_value *= cum_return
                # Fill portfolio history for the period
                self.portfolio_history.loc[last_date:event_date, 'portfolio_value'] = np.linspace(
                    self.portfolio_history.loc[last_date, 'portfolio_value'], current_value, len(period)
                )
            
            # At the event, adjust the weight for the ticker based on action
            # For simplicity, we adjust the weight by a fixed increment/decrement, e.g., 5%
            adjustment = 0.05 * action
            # Update the ticker weight, ensuring it stays between 0 and 1
            portfolio_weights[ticker] = np.clip(portfolio_weights[ticker] + adjustment, 0, 1)
            
            # After rebalancing, normalize weights to sum to 1
            total_weight = sum(portfolio_weights.values())
            portfolio_weights = {k: v/total_weight for k, v in portfolio_weights.items()}
            
            # Simulate performance over the holding period after the event
            event_index = self.price_data.index.get_loc(event_date)
            # Determine the end index for the holding period (ensuring we don't exceed available data)
            end_index = min(event_index + self.holding_period, len(self.price_data.index) - 1)
            holding_dates = self.price_data.index[event_index:end_index+1]
            period = self.price_data.loc[holding_dates]
            if not period.empty:
                returns = period.pct_change().fillna(0)
                daily_portfolio_return = returns.dot(pd.Series(portfolio_weights))
                cum_return = np.prod(1 + daily_portfolio_return)
                current_value *= cum_return
                self.portfolio_history.loc[holding_dates, 'portfolio_value'] = np.linspace(
                    current_value / cum_return, current_value, len(period)
                )
                # Update last_date for next event
                last_date = holding_dates[-1]
        
        # Fill any remaining portfolio history forward using last known portfolio weight returns
        if last_date < self.price_data.index[-1]:
            period = self.price_data.loc[last_date:]
            returns = period.pct_change().fillna(0)
            daily_portfolio_return = returns.dot(pd.Series(portfolio_weights))
            cum_return = np.prod(1 + daily_portfolio_return)
            current_value *= cum_return
            self.portfolio_history.loc[last_date:, 'portfolio_value'] = np.linspace(
                self.portfolio_history.loc[last_date, 'portfolio_value'], current_value, len(period)
            )
        
        # Drop rows with NaN values (if any)
        self.portfolio_history.dropna(inplace=True)
        return self.portfolio_history

    def compute_sharpe_ratio(self, risk_free_rate=0.02, annualize_factor=252):
        # Compute daily returns from the portfolio history
        self.portfolio_history['daily_return'] = self.portfolio_history['portfolio_value'].pct_change().fillna(0)
        # Excess returns over the risk-free rate (daily rate assumed)
        daily_rf = (risk_free_rate / annualize_factor)
        excess_returns = self.portfolio_history['daily_return'] - daily_rf
        avg_excess_return = excess_returns.mean()
        std_excess_return = excess_returns.std()
        sharpe_ratio = (avg_excess_return / std_excess_return) * np.sqrt(annualize_factor)
        return sharpe_ratio

class PortfolioAnalyzer:
    def __init__(self, portfolio_history, risk_free_rate=0.02, annualize_factor=252):
        self.portfolio_history = portfolio_history
        self.risk_free_rate = risk_free_rate
        self.annualize_factor = annualize_factor
    
    def compute_sharpe_ratio(self):
        daily_returns = self.portfolio_history['portfolio_value'].pct_change().fillna(0)
        daily_rf = (self.risk_free_rate / self.annualize_factor)
        excess_returns = daily_returns - daily_rf
        return (excess_returns.mean() / excess_returns.std()) * np.sqrt(self.annualize_factor)
    
    @staticmethod
    def benchmark_buy_and_hold(price_data, initial_value=1.0):
        eq_weights = np.repeat(1.0/price_data.shape[1], price_data.shape[1])
        daily_returns = price_data.pct_change().fillna(0)
        portfolio_returns = daily_returns.dot(eq_weights)
        return initial_value * np.cumprod(1 + portfolio_returns)


def main():
    tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'FB']
    data_gen = DataGenerator(tickers, '2022-01-01', '2024-12-31')
    
    price_data = data_gen.generate_price_data()
    events_df = data_gen.generate_events()
    
    backtester = EventDrivenBacktest(price_data, events_df, holding_period=5)
    portfolio_history = backtester.run()
    
    analyzer = PortfolioAnalyzer(portfolio_history)
    strategy_sharpe = analyzer.compute_sharpe_ratio()
    benchmark_values = analyzer.benchmark_buy_and_hold(price_data)
    
    benchmark_sharpe = (
        (benchmark_values.pct_change().mean() / benchmark_values.pct_change().std()) 
        * np.sqrt(252)
    )
    
    print(f"Strategy Sharpe Ratio: {strategy_sharpe:.2f}")
    print(f"Benchmark Sharpe Ratio: {benchmark_sharpe:.2f}")
    
    plt.figure(figsize=(10, 6))
    plt.plot(price_data.index, benchmark_values, label='Buy-and-Hold Equal Weight')
    plt.plot(portfolio_history.index, portfolio_history['portfolio_value'], 
            label='Event-Driven Strategy')
    plt.xlabel('Date')
    plt.ylabel('Portfolio Value')
    plt.title('Backtest Comparison')
    plt.legend()
    plt.grid(True)
    plt.show()

if __name__ == "__main__":
    main()
